### Importando bibliotecas

In [4]:
import pprint
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from itertools import batched

In [6]:
# Carregando os datasets
df_train_loaded = pd.read_csv("arquivos_csv/train_dataset1.csv")
df_test_loaded = pd.read_csv("arquivos_csv/test_dataset1.csv")


# Separando os dados de treinamento
X_train = df_train_loaded.drop("label", axis=1).values      # (n_features, n_amostras)
y_train = df_train_loaded["label"].values.reshape(1, -1).T    # (1, n_amostras)

# Separando os dados de teste
X_test = df_test_loaded.drop("label", axis=1).values
y_test = df_test_loaded["label"].values.reshape(1, -1).T

### Função de treinamento

In [ ]:
def treinamento(X_train, y_train, qtd_atributos, tx_aprendizagem, epocas, batch, precisao=1e-6, seed=None):
    """
        Argumentos:
        X_train -- matriz de entrada contendo as amostras de treino
        y_train -- vetor contendo os rótulos desejados para as amostras de treino
        qtd_atributos -- número de (features) presentes nas amostras
        tx_aprendizagem -- taxa de aprendizado, define o passo da atualização dos pesos
        epocas -- número máximo de ciclos completos sobre o conjunto de treinamento
        batch -- se True, o treinamento ocorre por lotes e se None, o treinamento ocorre por amostra
        precisao -- critério de parada baseado na diferença entre EQM de épocas consecutivas
        seed -- semente para inicialização dos pesos aleatórios
        
        Saída:
        parâmetros -- python dict contendo:
                        pesos -- vetor de pesos ajustados após o treinamento
                        epocas_executadas -- número total de épocas percorridas até a convergência
                        historico_erro -- lista com a evolução do erro de classificação ao longo das épocas
    """

    if seed is not None:                            # iniciando o vetor de pesos
        num_aleatorio = np.random.seed(seed)
        pesos = num_aleatorio.random.uniform(low=-1, high=1, size=((qtd_atributos + 1),))   # vetor de peso (atributos + bias)
    else:
        pesos = np.random.uniform(low=-1, high=1, size=((qtd_atributos + 1),))              # vetor de peso (atributos + bias)

    X = np.c_[np.ones(len(X_train)), X_train]
    y = y_train.flatten()                           # garantindo que é um vetor unidimensional

    evolucao_erro = []
    epoca = 0

    while epoca < epocas:
        erro = 0

        if epoca > 0:
            EQM_anterior = np.mean((y - np.dot(X, pesos))**2)
        else:
            EQM_anterior = 1e6              # tratamento para a primeira epoca
        
        if batch:                           # treinamento por lote (batch)
            u = np.dot(X, pesos)            # potencial de ativação
            erro = y - u
            pesos += tx_aprendizagem * np.dot(X.T, erro)
        
        else:                               # treinamento por amostra
            for amostra in range(len(X)):
                u = np.dot(pesos, X[amostra])   # potencial de ativação
                erro = y[amostra] - u
                pesos += tx_aprendizagem * erro * X[amostra]
        
        epoca += 1
        
        EQM_atual = np.mean((y - np.dot(X, pesos))**2)
        
        predicoes = np.where(np.dot(X, pesos) >= 0, 1, -1)
        erro = np.count_nonzero(predicoes - y)
        evolucao_erro.append(erro/len(y))
        
        if np.abs(EQM_atual - EQM_anterior) <= precisao:
            break
    
    return pesos, epoca, evolucao_erro